In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

2.3.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 45.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os.path as osp
import os
import torch
import torch_geometric
from torch_geometric.datasets import AMiner
from torch_geometric.nn import MetaPath2Vec
import torch_geometric.transforms as T


In [3]:
path = r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\metapath2vec\Aminer'
dataset = AMiner(path)
data = dataset[0]

Extracting C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\metapath2vec\Aminer/net_aminer.zip
Extracting C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\metapath2vec\Aminer/raw/label.zip
Processing...
Done!


In [25]:
reverse_edge_types = [(x[2], "{}_{}".format('rev', x[1]), x[0]) for x in data.edge_types]

transform = T.RandomLinkSplit(
    num_val=0.1,  # fraction of data held out for validation
    num_test=0.8, # fraction of data held out for test
    neg_sampling_ratio=0.0,
    edge_types = data.edge_types,
    rev_edge_types= reverse_edge_types
)
train_data, val_data, test_data = transform(data)

In [26]:
print(train_data)
print('')
print(f'{val_data}')
print('')
print(test_data)


HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={
    edge_index=[2, 932361],
    edge_label=[932361],
    edge_label_index=[2, 932361],
  },
  (author, writes, paper)={
    edge_index=[2, 932361],
    edge_label=[932361],
    edge_label_index=[2, 932361],
  },
  (paper, published_in, venue)={
    edge_index=[2, 319441],
    edge_label=[319441],
    edge_label_index=[2, 319441],
  },
  (venue, publishes, paper)={
    edge_index=[2, 319441],
    edge_label=[319441],
    edge_label_index=[2, 319441],
  },
  (author, rev_written_by, paper)={},
  (paper, rev_writes, author)={},
  (venue, rev_published_in, paper)={},
  (paper, rev_publishes, venue)={}
)

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nod

In [16]:
data.edge_index_dict


{('paper',
  'written_by',
  'author'): tensor([[      0,       1,       2,  ..., 3194404, 3194404, 3194404],
         [      0,       1,       2,  ...,    4393,   21681,  317436]]),
 ('author',
  'writes',
  'paper'): tensor([[      0,       1,       2,  ...,    4393,   21681,  317436],
         [      0,       1,       2,  ..., 3194404, 3194404, 3194404]]),
 ('paper',
  'published_in',
  'venue'): tensor([[      0,       1,       2,  ..., 3194402, 3194403, 3194404],
         [   2190,    2190,    2190,  ...,    3148,    3148,    3148]]),
 ('venue',
  'publishes',
  'paper'): tensor([[   2190,    2190,    2190,  ...,    3148,    3148,    3148],
         [      0,       1,       2,  ..., 3194402, 3194403, 3194404]])}

In [18]:

# Print the overall structure of the dataset
print("Overall structure of the dataset:")
print(data)

# List all node types and their features
print("\nNode types and their features:")
for node_type in data.node_types:
    print(f"Node type: {node_type}")
    for key, value in data[node_type].items():
        print(f"  {key}: {value}")

# List all edge types and their indices
print("\nEdge types and their indices:")
for edge_type in data.edge_types:
    print(f"Edge type: {edge_type}")
    print(f"  Edge index shape: {data[edge_type].edge_index.shape}")

# Example of accessing specific node features
print("\nExample of accessing specific node features:")
author_features = data['author']
print("Author features:", author_features)

# Example of accessing specific edge indices
print("\nExample of accessing specific edge indices:")
author_paper_edge_index = data['author', 'writes', 'paper'].edge_index
print("Author writes paper edge index:", author_paper_edge_index)


Overall structure of the dataset:
HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)

Node types and their features:
Node type: author
  y: tensor([0, 2, 5,  ..., 0, 1, 5])
  y_index: tensor([ 168866, 1327323,     870,  ...,  168759,  254769,  264374])
  num_nodes: 1693531
Node type: venue
  y: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
        5, 5, 5,

In [4]:
metapath = [
    ('author', 'writes', 'paper'),
    ('paper', 'published_in', 'venue'),
    ('venue', 'publishes', 'paper'),
    ('paper', 'written_by', 'author'),
]

if torch.cuda.is_available():
    print('Cuda')
    device = torch.device('cuda')
elif torch_geometric.is_xpu_available():
    print('xpu')
    device = torch.device('xpu')
else:
    print('cpu')
    device = torch.device('cpu')

model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=50, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=6)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)



Cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:

def train(epoch, log_steps=1000, eval_steps=2000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            author_acc, venue_acc= test()
            print('')
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Author Acc: {author_acc:.4f}'
                   f'Venue Acc: {venue_acc:.4f}'))

def node_test(node_class, train_ratio=0.1):
    z = model(node_class, batch=data[node_class].y_index.to(device))
    y = data[node_class].y

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)


@torch.no_grad()
def test(train_ratio=0.1):
    model.eval()

    author_acc = node_test('author')
    venue_acc = node_test('venue')

    return author_acc, venue_acc



In [9]:

for epoch in range(1, 6):
    train(epoch)
    # acc = test()
    # print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 1, Step: 01000/13231, Loss: 2.3792
Epoch: 1, Step: 02000/13231, Loss: 2.0392

Epoch: 1, Step: 02000/13231, Author Acc: 0.6363Venue Acc: 0.4793
Epoch: 1, Step: 03000/13231, Loss: 1.7971
Epoch: 1, Step: 04000/13231, Loss: 1.6216

Epoch: 1, Step: 04000/13231, Author Acc: 0.7387Venue Acc: 0.5289
Epoch: 1, Step: 05000/13231, Loss: 1.4902
Epoch: 1, Step: 06000/13231, Loss: 1.3879

Epoch: 1, Step: 06000/13231, Author Acc: 0.7920Venue Acc: 0.6942
Epoch: 1, Step: 07000/13231, Loss: 1.3067
Epoch: 1, Step: 08000/13231, Loss: 1.2403

Epoch: 1, Step: 08000/13231, Author Acc: 0.8246Venue Acc: 0.6198
Epoch: 1, Step: 09000/13231, Loss: 1.1855
Epoch: 1, Step: 10000/13231, Loss: 1.1401

Epoch: 1, Step: 10000/13231, Author Acc: 0.8446Venue Acc: 0.8182
Epoch: 1, Step: 11000/13231, Loss: 1.1019
Epoch: 1, Step: 12000/13231, Loss: 1.0695

Epoch: 1, Step: 12000/13231, Author Acc: 0.8591Venue Acc: 0.4959
Epoch: 1, Step: 13000/13231, Loss: 1.0417


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 2, Step: 01000/13231, Loss: 1.0126
Epoch: 2, Step: 02000/13231, Loss: 0.9930

Epoch: 2, Step: 02000/13231, Author Acc: 0.8739Venue Acc: 0.8678
Epoch: 2, Step: 03000/13231, Loss: 0.9761
Epoch: 2, Step: 04000/13231, Loss: 0.9614

Epoch: 2, Step: 04000/13231, Author Acc: 0.8802Venue Acc: 0.7190
Epoch: 2, Step: 05000/13231, Loss: 0.9485
Epoch: 2, Step: 06000/13231, Loss: 0.9373

Epoch: 2, Step: 06000/13231, Author Acc: 0.8850Venue Acc: 0.6612
Epoch: 2, Step: 07000/13231, Loss: 0.9276
Epoch: 2, Step: 08000/13231, Loss: 0.9189

Epoch: 2, Step: 08000/13231, Author Acc: 0.8895Venue Acc: 0.8099
Epoch: 2, Step: 09000/13231, Loss: 0.9110
Epoch: 2, Step: 10000/13231, Loss: 0.9041

Epoch: 2, Step: 10000/13231, Author Acc: 0.8919Venue Acc: 0.7438
Epoch: 2, Step: 11000/13231, Loss: 0.8982
Epoch: 2, Step: 12000/13231, Loss: 0.8926

Epoch: 2, Step: 12000/13231, Author Acc: 0.8958Venue Acc: 0.5372
Epoch: 2, Step: 13000/13231, Loss: 0.8878


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 3, Step: 01000/13231, Loss: 0.8822
Epoch: 3, Step: 02000/13231, Loss: 0.8783

Epoch: 3, Step: 02000/13231, Author Acc: 0.8994Venue Acc: 0.5785
Epoch: 3, Step: 03000/13231, Loss: 0.8748
Epoch: 3, Step: 04000/13231, Loss: 0.8716

Epoch: 3, Step: 04000/13231, Author Acc: 0.9007Venue Acc: 0.6942
Epoch: 3, Step: 05000/13231, Loss: 0.8688
Epoch: 3, Step: 06000/13231, Loss: 0.8660

Epoch: 3, Step: 06000/13231, Author Acc: 0.9019Venue Acc: 0.5207
Epoch: 3, Step: 07000/13231, Loss: 0.8636
Epoch: 3, Step: 08000/13231, Loss: 0.8614

Epoch: 3, Step: 08000/13231, Author Acc: 0.9045Venue Acc: 0.6942
Epoch: 3, Step: 09000/13231, Loss: 0.8593
Epoch: 3, Step: 10000/13231, Loss: 0.8574

Epoch: 3, Step: 10000/13231, Author Acc: 0.9056Venue Acc: 0.6198
Epoch: 3, Step: 11000/13231, Loss: 0.8556
Epoch: 3, Step: 12000/13231, Loss: 0.8538

Epoch: 3, Step: 12000/13231, Author Acc: 0.9078Venue Acc: 0.4793
Epoch: 3, Step: 13000/13231, Loss: 0.8525


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 4, Step: 01000/13231, Loss: 0.8504
Epoch: 4, Step: 02000/13231, Loss: 0.8492

Epoch: 4, Step: 02000/13231, Author Acc: 0.9097Venue Acc: 0.7025
Epoch: 4, Step: 03000/13231, Loss: 0.8480
Epoch: 4, Step: 04000/13231, Loss: 0.8468

Epoch: 4, Step: 04000/13231, Author Acc: 0.9109Venue Acc: 0.7521
Epoch: 4, Step: 05000/13231, Loss: 0.8458
Epoch: 4, Step: 06000/13231, Loss: 0.8448

Epoch: 4, Step: 06000/13231, Author Acc: 0.9109Venue Acc: 0.7603
Epoch: 4, Step: 07000/13231, Loss: 0.8440
Epoch: 4, Step: 08000/13231, Loss: 0.8431

Epoch: 4, Step: 08000/13231, Author Acc: 0.9122Venue Acc: 0.5207
Epoch: 4, Step: 09000/13231, Loss: 0.8423
Epoch: 4, Step: 10000/13231, Loss: 0.8415

Epoch: 4, Step: 10000/13231, Author Acc: 0.9142Venue Acc: 0.8264
Epoch: 4, Step: 11000/13231, Loss: 0.8408
Epoch: 4, Step: 12000/13231, Loss: 0.8400

Epoch: 4, Step: 12000/13231, Author Acc: 0.9150Venue Acc: 0.5785
Epoch: 4, Step: 13000/13231, Loss: 0.8394


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 5, Step: 01000/13231, Loss: 0.8384
Epoch: 5, Step: 02000/13231, Loss: 0.8380

Epoch: 5, Step: 02000/13231, Author Acc: 0.9156Venue Acc: 0.6529
Epoch: 5, Step: 03000/13231, Loss: 0.8376
Epoch: 5, Step: 04000/13231, Loss: 0.8371

Epoch: 5, Step: 04000/13231, Author Acc: 0.9167Venue Acc: 0.8678
Epoch: 5, Step: 05000/13231, Loss: 0.8366
Epoch: 5, Step: 06000/13231, Loss: 0.8361

Epoch: 5, Step: 06000/13231, Author Acc: 0.9166Venue Acc: 0.7438
Epoch: 5, Step: 07000/13231, Loss: 0.8357
Epoch: 5, Step: 08000/13231, Loss: 0.8354

Epoch: 5, Step: 08000/13231, Author Acc: 0.9169Venue Acc: 0.6529
Epoch: 5, Step: 09000/13231, Loss: 0.8351
Epoch: 5, Step: 10000/13231, Loss: 0.8346

Epoch: 5, Step: 10000/13231, Author Acc: 0.9179Venue Acc: 0.5041
Epoch: 5, Step: 11000/13231, Loss: 0.8344
Epoch: 5, Step: 12000/13231, Loss: 0.8341

Epoch: 5, Step: 12000/13231, Author Acc: 0.9186Venue Acc: 0.4628
Epoch: 5, Step: 13000/13231, Loss: 0.8339


In [11]:
data['author']

tensor([ 168866, 1327323,     870,  ...,  168759,  254769,  264374])

In [5]:
data['venue']

{'y': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]), 'y_index': tensor([1741, 2245,  111,  837, 2588, 2116, 2696, 3648, 3784,  313, 3414,  598,
        2995, 2716, 1423,  783, 1902, 3132, 1753, 2748, 2660, 3182,  775, 3339,
        1601, 3589,  156, 1145,  692, 3048,  925, 1587,  820, 1374, 3719,  819,
         492, 3830, 2777, 3001, 3693,  517, 1808, 2353, 3499, 1763, 2372, 1030,
         721, 2680, 3355, 1217, 3400, 1271, 1970, 1127,  407,  353, 1471, 1095,
         477, 3701,   65, 1009, 1899, 1442, 2073, 3143, 2466,  289, 1996, 1070,
        3871, 3695,  281, 3633,   50, 2642, 1925, 1

In [6]:
data['paper']

{'num_nodes': 3194405}

In [ ]:
10% training,10%validation, 80% test, splitting the training mask and split evenly by the label
DBLP hetero
LastFM
author node; venue nodes; paper nodes

run DBLP and LastFM the same splits

heterogenous graph masked autoencoders
git hub look for it. reimplement model,

1. drug traffickers is a big problem in united states and in social media
2. dealers people who follow them, freinds are high likely to be drug traffickers

2. limited dataset to train drug trafficking model. Manually labeled and annotated dataset,
3. build model to handle inbalanced data, bc pos users are much smaller than neg users.


RUn/gat/gin on hetergoenous graphs ie. aminer, dblp and lastfm.

